In [1]:
import os.path
import time

import torch
torch.cuda.empty_cache()

# Imports


In [2]:
from src.efficentnet_train import data_load,models_train,visualization,utils,generate_dataset,evaluation
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from src.efficentfacenet import face_descriptor
from torchvision import transforms
import pandas as pd



In [3]:
dataset_path = "../dataset/preprocessed"
model_weights_path="../model_weights/training"
train_data_save_path="../training log"
batch_size=8
stop_n_layers=5

In [4]:
train_history=pd.read_csv(train_data_save_path+"/train_data.csv",index_col=False)

In [4]:
train_history

,Train Loss,no train rows,Test Loss,No test rows,Time taken (M),Notes,Date,Time
0,0.280173,100000,0.342320,25000,65.2,remove sigmoid from classifier + input normali...,15/05/2022,23:26:00
1,0.989897,100000,1.000320,25000,60.1,remove sigmoid from classifier + input normali...,16/05/2022,0:27:00
2,1.000000,100000,1.000320,25000,60.2,remove sigmoid from classifier + input normali...,16/05/2022,1:27:00
3,0.173325,50000,0.284327,10000,63.0,remove sigmoid from classifier + input normali...,16/05/2022,19:28:00
4,0.243919,50000,0.281481,10000,60.0,remove sigmoid from classifier + input normali...,16/05/2022,20:28:00
...,...,...,...,...,...,...,...,...
141,0.099156,10000,0.173237,2500,18.4,"Classifier -->(1280,256,128)+ full train , ran...",21/05/2022,11:59:00
142,0.098474,10000,0.151633,2500,19.1,"Classifier -->(1280,256,128)+ full train , ran...",21/05/2022,12:19:00
143,0.093220,10000,0.161684,2500,20.1,"Classifier -->(1280,256,128)+ full train , ran...",21/05/2022,12:39:00
144,0.099141,10000,0.121382,2500,20.0,"Classifier -->(1280,256,128)+ full train , ran...",21/05/2022,12:59:00


# Load Data

In [4]:
train_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((240, 240)),transforms.RandomHorizontalFlip(),transforms.RandomRotation(10),transforms.RandomAutocontrast(),data_load.Normalize()])
test_transform=transforms.Compose([transforms.ToTensor(), transforms.Resize((240, 240)),data_load.Normalize()])

# Face descriptor model train

In [ ]:
train_dataset = data_load.FacesTripletDataset(f"{dataset_path}/train", 5000, train_transform,True)
test_dataset=data_load.FacesTripletDataset(f"{dataset_path}/test", 1000, test_transform,True)


 5.581395 % of the folders processederror loading --> ../dataset/preprocessed/train\8563\072956.jpg
 6.395348 % of the folders processed

In [7]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
ts=time.time()
a_batch, p_batch, n_batch = next(iter(train_loader))
print(time.time()-ts)
for i in range(batch_size):
    fig, ax = plt.subplots(1, 3)
    ax[0].set_title("anchor")
    ax[0].imshow(a_batch[i].numpy().transpose([1,2,0]))


    ax[1].set_title("positive")
    ax[1].imshow(p_batch[i].numpy().transpose([1,2,0]))



    ax[2].set_title("negative")
    ax[2].imshow(n_batch[i].numpy().transpose([1,2,0]))



In [3]:
model = face_descriptor.FaceDescriptorModel(download_weights=True, version="efficientnet_b1")



In [7]:
model

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [8]:
for i in range (len(model.features)):
      for parm in model.features[i].parameters():
            parm.requires_grad=True

In [19]:
for i in range(stop_n_layers):
  for parm in model.features[i].parameters():
    parm.requires_grad=True


In [11]:
epochs=15
learning_rate=0.001
train_notes="Classifier -->(1280,256,128)+ full train , randomHorizontal flip,random Rotation , randomAutoContrast,effnetb1 , learnrate =0.0001"


In [9]:
model.load_local_weights("../model_weights/training/05_20 19_05 Train_(0.160565) Test_(0.166811).pt",True)

In [27]:
train_losses=models_train.model_train(model,epochs=1000,learn_rate=0.0001,train_loader=train_loader,test_loader=test_loader,train_mod="triplet",cuda=True,weight_saving_path=model_weights_path,epoch_data_saving_path=train_data_save_path,notes=train_notes)

Testing before training ...
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.150402Test Loss before Training=0.15064265232086183
 epoch 1 [==========] time remaining = 0.0 Avg Train_Loss=0.09595096027
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.15975860270
 epoch 1 train_loss =0.09595097725391388 test_loss=0.16001451511383058
!!!Warning Overfitting!!!
 epoch 2 [==========] time remaining = 0.0 Avg Train_Loss=0.09915699235
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.17296072805
 epoch 2 train_loss =0.09915616257190704 test_loss=0.17323749313354492
!!!Warning Overfitting!!!
 epoch 3 [==========] time remaining = 0.0 Avg Train_Loss=0.09847398552
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.15139051876
 epoch 3 train_loss =0.09847376960515976 test_loss=0.15163300457000733
!!!Warning Overfitting!!!
 epoch 4 [==========] time remaining = 0.0 Avg Train_Loss=0.09322093294
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.16142

KeyboardInterrupt: 

In [14]:
model.load_local_weights(model_weights_path+"/05_19 05_11 Train_(0.203699) Test_(0.367653).pt",cuda_weights=True)

# Face descriptor evaluation

In [13]:
features_dict=data_load.get_pic_features_dict(f"{dataset_path}/test",model,test_transform,cuda=True)

 data processed [==========] time remaing=0.006

In [14]:
test_data_pd=pd.DataFrame(generate_dataset.generate_testing_data_set_frame(dataset_path+"/test",True))

In [26]:
evaluation.model_test(features_dict,test_data_pd,results_path="",threshold=6.3)

 Testing  [==========] time remaining = 0.0 Accuracy =86.064Accuracy now equal --> 86.0645%


(                              Mertic         Value
 0                     processed rows  88192.000000
 1  Model accuracy on Proceed Faces %     86.064000
 2                     False Positive  10922.000000
 3                     False Negative   1368.000000
 4                          precision      0.796406
 5                             recall      0.968955
 6                        fbeta-score      0.444916
 7           avg same person distance      3.747763
 8           avg diff person distance      8.288865
 9                    Model tolerance      6.300000,
                  Actual True  Actual False
 Predicted True         42728         10922
 Predicted False         1368         33174)

# Efficient-Facenet model train
- the trained descriptor + classifier

In [5]:
descriptor_weights="../model_weights/training/05_21 12_59 Train_(0.099141) Test_(0.121381).pt"

In [6]:
pair_train_dataset=data_load.FacesPairDataset(f"{dataset_path}/train",5000,train_transform,load_imgs_from_dict=True)
pair_test_dataset=data_load.FacesPairDataset(f"{dataset_path}/test",1000,train_transform,load_imgs_from_dict=True)

 100.0 % of the folders processed img dict loaded in 0.47 m
 100.0 % of the folders processed img dict loaded in 0.1 m


In [7]:
pair_train_loader=DataLoader(pair_train_dataset,batch_size,shuffle=False)
pair_test_loader=DataLoader(pair_test_dataset,batch_size,shuffle=False)

In [8]:
itr=iter(pair_train_dataset)

In [29]:
face_x,face_y,label=next(itr)

In [30]:
face_x.shape

torch.Size([3, 240, 240])

In [9]:
efficient_facenet=face_descriptor.EfficientFacenet(128,descriptor_weights)

In [10]:
for param in efficient_facenet.descriptor.parameters():
    param.requires_grad=False

In [11]:
train_notes="efficient facenet classifier train test "
model_weights_path="../model_weights/efficient_facenet"

In [34]:
models_train.model_train(efficient_facenet,10,0.01,pair_train_loader,pair_test_loader,"pair",cuda=False,weight_saving_path=model_weights_path,epoch_data_saving_path=train_data_save_path,notes=train_notes)

Testing before training ...
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.086479Test Loss before Training=0.08647927123308181
 epoch 1 [==========] time remaining = 0.0 Avg Train_Loss=0.04633346293
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.07999680217
 epoch 1 train_loss =0.04633351444860455 test_loss=0.07999646864458919
new minimum test loss 0.079996  achieved, model weights saved 
!!!Warning Overfitting!!!
 epoch 2 [==========] time remaining = 0.0 Avg Train_Loss=0.07147071445
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.08292482894
 epoch 2 train_loss =0.07147001727577298 test_loss=0.08292476823925972
!!!Warning Overfitting!!!
 epoch 3 [==========] time remaining = 0.0 Avg Train_Loss=0.08687386874
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.08664386643
 epoch 3 train_loss =0.08687373447418213 test_loss=0.08664339780807495
 epoch 4 [==========] time remaining = 0.0 Avg Train_Loss=0.08664386643
 Testing  [==========] time 

([0.04633351444860455,
  0.07147001727577298,
  0.08687373447418213,
  0.08664339780807495,
  0.08664339780807495,
  0.08664339780807495,
  0.08664339780807495,
  0.08664339780807495,
  0.08664962853193284,
  0.08664339780807495],
 [0.07999646864458919,
  0.08292476823925972,
  0.08664339780807495,
  0.08664339780807495,
  0.08664339780807495,
  0.08664339780807495,
  0.08664339780807495,
  0.08664339780807495,
  0.08664339780807495,
  0.08664339780807495])

In [13]:
efficient_facenet.load_local_weights("../model_weights/efficient_facenet/05_24 04_42 Train_(0.046333) Test_(0.079996).pt")

In [ ]:
imgs_features_dict=data_load.get_pic_features_dict(f"{dataset_path}/test",model=efficient_facenet.descriptor,transform=test_transform,cuda=False)